In [1]:
import pandas as pd
import numpy as np
import csv
import os
import hdbscan
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [31]:
header = ['Filename','Selection', 'View', 'Channel', 'Begin Time (s)', 'End Time (s)', 'Low Freq (Hz)', 'High Freq (Hz)', 'Species Code', 'Common Name', 'Confidence']
data =np.matrix(header)

entries = os.scandir('./output/large-classify')
for entry in entries:
    if (entry.is_file()):
        path = entry.path
        f = open(path, "r")
        classifications = f.read().strip().split("\n")
        name = entry.name
        for text in classifications:
            text = text.split("\t")
            text.insert(0, entry.name.split(".")[0])
            if ('Selection' not in text):
                data = np.append(data, [text],axis=0)
            
data

matrix([['Filename', 'Selection', 'View', ..., 'Species Code',
         'Common Name', 'Confidence'],
        ['Lipaugus-vociferans-101', '1', 'Spectrogram 1', ..., 'blfant1',
         'Black-faced Antthrush', '0.3000'],
        ['Lipaugus-vociferans-101', '2', 'Spectrogram 1', ..., 'scrpih1',
         'Screaming Piha', '0.1527'],
        ...,
        ['XC552343 - Screaming Piha - Lipaugus vociferans', '2',
         'Spectrogram 1', ..., 'scrpih1', 'Screaming Piha', '0.9971'],
        ['XC552343 - Screaming Piha - Lipaugus vociferans', '3',
         'Spectrogram 1', ..., 'blttro2', 'Black-throated Trogon',
         '0.1040'],
        ['XC591821 - Screaming Piha - Lipaugus vociferans', '1',
         'Spectrogram 1', ..., 'scrpih1', 'Screaming Piha', '0.1795']],
       dtype='<U47')

In [32]:
classifications = pd.DataFrame(data = data)
new_header = classifications.iloc[0] #grab the first row for the header
classifications = classifications[1:] #take the data less the header row
classifications.columns = new_header #set the header row as the df header
#https://www.codegrepper.com/code-examples/python/pandas+set+first+row+as+column+names
classifications

,Filename,Selection,View,Channel,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Species Code,Common Name,Confidence
1,Lipaugus-vociferans-101,1,Spectrogram 1,1,12.0,15.0,150,12000,blfant1,Black-faced Antthrush,0.3000
2,Lipaugus-vociferans-101,2,Spectrogram 1,1,15.0,18.0,150,12000,scrpih1,Screaming Piha,0.1527
3,Lipaugus-vociferans-101,3,Spectrogram 1,1,21.0,24.0,150,12000,scrpih1,Screaming Piha,0.5965
4,Lipaugus-vociferans-101,4,Spectrogram 1,1,21.0,24.0,150,12000,blfant1,Black-faced Antthrush,0.1684
5,Lipaugus-vociferans-101,5,Spectrogram 1,1,21.0,24.0,150,12000,rucant2,Rufous-capped Antthrush,0.1051
...,...,...,...,...,...,...,...,...,...,...,...
5105,XC425555 - Screaming Piha - Lipaugus vociferans,7,Spectrogram 1,1,9.0,12.0,150,12000,scrpih1,Screaming Piha,0.2043
5106,XC552343 - Screaming Piha - Lipaugus vociferans,1,Spectrogram 1,1,0,3.0,150,12000,scrpih1,Screaming Piha,0.9997
5107,XC552343 - Screaming Piha - Lipaugus vociferans,2,Spectrogram 1,1,3.0,6.0,150,12000,scrpih1,Screaming Piha,0.9971
5108,XC552343 - Screaming Piha - Lipaugus vociferans,3,Spectrogram 1,1,3.0,6.0,150,12000,blttro2,Black-throated Trogon,0.1040


In [35]:
#classifications.groupby(["Filename", 'Begin Time (s)', 'End Time (s)']).min()
classifications.to_csv("./classifications.csv")

In [40]:
duplicated = classifications.duplicated(subset=["Filename", 'Begin Time (s)', 'End Time (s)'])
duplicated_df  = classifications[duplicated].sort_values(by="Confidence", ascending=False)
duplicated_df

,Filename,Selection,View,Channel,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Species Code,Common Name,Confidence
1095,Lipaugus-vociferans-22528,2,Spectrogram 1,1,0,3.0,150,12000,piecur1,Pied Currawong,0.8790
3133,Lipaugus-vociferans-450614,4,Spectrogram 1,1,15.0,18.0,150,12000,scrpih1,Screaming Piha,0.8507
4045,Lipaugus-vociferans-525378,21,Spectrogram 1,1,48.0,51.0,150,12000,blcman1,Blue-crowned Manakin,0.8454
993,Lipaugus-vociferans-225161,2,Spectrogram 1,1,0,3.0,150,12000,scrpih1,Screaming Piha,0.7826
1750,Lipaugus-vociferans-329364,12,Spectrogram 1,1,114.0,117.0,150,12000,scrpih1,Screaming Piha,0.7705
...,...,...,...,...,...,...,...,...,...,...,...
3711,Lipaugus-vociferans-519394,22,Spectrogram 1,1,54.0,57.0,150,12000,blttro2,Black-throated Trogon,0.1003
1273,Lipaugus-vociferans-249182,4,Spectrogram 1,1,3.0,6.0,150,12000,retcar2,Red-throated Caracara,0.1003
378,Lipaugus-vociferans-129738,34,Spectrogram 1,1,63.0,66.0,150,12000,grejac2,Great Jacamar,0.1003
4009,Lipaugus-vociferans-523948,28,Spectrogram 1,1,57.0,60.0,150,12000,cintin1,Cinereous Tinamou,0.1001


In [44]:
duplicated_df

,Filename,Selection,View,Channel,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Species Code,Common Name,Confidence
1095,Lipaugus-vociferans-22528,2,Spectrogram 1,1,0,3.0,150,12000,piecur1,Pied Currawong,0.8790
3133,Lipaugus-vociferans-450614,4,Spectrogram 1,1,15.0,18.0,150,12000,scrpih1,Screaming Piha,0.8507
4045,Lipaugus-vociferans-525378,21,Spectrogram 1,1,48.0,51.0,150,12000,blcman1,Blue-crowned Manakin,0.8454
993,Lipaugus-vociferans-225161,2,Spectrogram 1,1,0,3.0,150,12000,scrpih1,Screaming Piha,0.7826
1750,Lipaugus-vociferans-329364,12,Spectrogram 1,1,114.0,117.0,150,12000,scrpih1,Screaming Piha,0.7705
...,...,...,...,...,...,...,...,...,...,...,...
5021,Lipaugus-vociferans-88597,16,Spectrogram 1,1,33.0,36.0,150,12000,speowl1,Spectacled Owl,0.1005
3167,Lipaugus-vociferans-456300,8,Spectrogram 1,1,21.0,24.0,150,12000,scrpih1,Screaming Piha,0.1004
378,Lipaugus-vociferans-129738,34,Spectrogram 1,1,63.0,66.0,150,12000,grejac2,Great Jacamar,0.1003
4009,Lipaugus-vociferans-523948,28,Spectrogram 1,1,57.0,60.0,150,12000,cintin1,Cinereous Tinamou,0.1001


In [47]:
classifications_no_duplicates = classifications.sort_values(by="Confidence", ascending=False).drop_duplicates(subset=["Filename", 'Begin Time (s)', 'End Time (s)'], keep='first', inplace=False)
classifications_no_duplicates

,Filename,Selection,View,Channel,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Species Code,Common Name,Confidence
4704,Lipaugus-vociferans-73778,1,Spectrogram 1,1,12.0,15.0,150,12000,scrpih1,Screaming Piha,1.0000
3030,Lipaugus-vociferans-443978,38,Spectrogram 1,1,192.0,195.0,150,12000,scrpih1,Screaming Piha,1.0000
2947,Lipaugus-vociferans-442394,8,Spectrogram 1,1,117.0,120.0,150,12000,scrpih1,Screaming Piha,1.0000
3012,Lipaugus-vociferans-443978,20,Spectrogram 1,1,147.0,150.0,150,12000,scrpih1,Screaming Piha,0.9999
4281,Lipaugus-vociferans-542536,3,Spectrogram 1,1,24.0,27.0,150,12000,scrpih1,Screaming Piha,0.9999
...,...,...,...,...,...,...,...,...,...,...,...
4289,Lipaugus-vociferans-543334,6,Spectrogram 1,1,24.0,27.0,150,12000,humnov,Human non-vocal,0.1013
1499,Lipaugus-vociferans-276152,2,Spectrogram 1,1,15.0,18.0,150,12000,leswoo2,Lesser Woodcreeper,0.1012
1612,Lipaugus-vociferans-301423,9,Spectrogram 1,1,24.0,27.0,150,12000,scrpih1,Screaming Piha,0.1011
3143,Lipaugus-vociferans-450616,4,Spectrogram 1,1,24.0,27.0,150,12000,gohman1,Golden-headed Manakin,0.1008


In [48]:
classifications_no_duplicates.to_csv("./classifications_no_duplicates.csv")